<a href="https://colab.research.google.com/github/hugogsmendes/Lab-Data-Science/blob/main/MultilayerPerceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
# Função de Ativação Sigmóide
def sigmoid (z):
  return 1.0/(1.0+np.exp(-z))

In [3]:
class Network (object):

  def __init__(self, sizes):
    self.num_layers = len(sizes) # Número de camadas
    self.sizes = sizes # Número de neurônios em cada camada
    self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
    self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

  def feedforward(self, a):
    for b, w in zip (self.biases, self.weights):
      a = sigmoid(np.dot(w, a) + b)
    return a